# TGE detection
The aim of this notebook is to develop an algorithm of detection TGE in Aragats data, namely "STAND3 coincidence 1000" series.

Requirements for running: a csv file with "timestamp" and "stand3_coincidence_1000" in ./data/Aragats/ folder.

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

## Load and plot data

In [ ]:
stand3_1000_df = pd.read_csv("../data/Aragats/stand3_coincidence_1000.csv")

In [ ]:
stand3_1000_df.head() 

In [ ]:
fig = go.Figure()
raw_data = go.Scatter(
    x=stand3_1000_df.timestamp,
    y=stand3_1000_df.STAND3_coincidence_1000,
    mode="lines",
    name="raw data"
    )
fig.add_traces(raw_data)
fig.update_layout(
    width=1200,
    height=700,
    title="STAND3 coincidence 1000"
)
fig.show()

## Smooth data

In [ ]:
def gaussian_weights(window_size, sigma):
    """
    Generate Gaussian weights for a given window size and standard deviation.
    
    Args:
    - window_size (int): The size of the moving window (must be odd).
    - sigma (float): The standard deviation of the Gaussian function.
    
    Returns:
    - weights (numpy array): Array of Gaussian weights.
    """
    # Create a range of indices centered around 0
    half_window = window_size // 2
    x = np.arange(-half_window, half_window + 1)
    
    # Compute the Gaussian weights
    weights = np.exp(-0.5 * (x / sigma) ** 2)
    
    # Normalize the weights so that they sum to 1
    weights /= np.sum(weights)
    
    return weights

def gaussian_moving_average(time_series, window_size, sigma):
    """
    Compute the moving average of a time series with Gaussian weights.
    
    Args:
    - time_series (array-like): The time series to smooth.
    - window_size (int): The size of the moving window (must be odd).
    - sigma (float): The standard deviation of the Gaussian function.
    
    Returns:
    - smoothed_series (numpy array): The smoothed time series.
    """
    # Convert the time series to a numpy array
    time_series = np.array(time_series)
    n = len(time_series)
    
    # Get Gaussian weights
    weights = gaussian_weights(window_size, sigma)
    
    # Initialize the output array
    smoothed_series = np.zeros_like(time_series)
    
    # Apply the moving average with Gaussian weights
    half_window = window_size // 2
    for i in range(n):
        # Define the window range
        start_idx = max(i - half_window, 0)
        end_idx = min(i + half_window + 1, n)
        
        # Get the corresponding slice of the time series
        window = time_series[start_idx:end_idx]
        
        # Get the Gaussian weights for the valid window
        valid_weights = weights[half_window - (i - start_idx):half_window + (end_idx - i)]
        
        # Compute the weighted average
        smoothed_series[i] = np.sum(window * valid_weights[:len(window)])  # Slice valid_weights to match window size
    
    return smoothed_series


In [ ]:
window_size = 61  # Odd number for symmetric window
sigma = 6.0  # Standard deviation for the Gaussian kernel

stand3_1000_df["gaussian_smoothed"] = gaussian_moving_average(stand3_1000_df.STAND3_coincidence_1000, window_size, sigma)

In [ ]:
fig = go.Figure()
raw_data = go.Scatter(
    x=stand3_1000_df.timestamp,
    y=stand3_1000_df.STAND3_coincidence_1000,
    mode="lines",
    name="raw data"
    )
smoothed_data_gauss = go.Scatter(
    x=stand3_1000_df.timestamp,
    y=stand3_1000_df.gaussian_smoothed,
    mode="lines",
    name="gauss smoothing"
    )
fig.add_traces(raw_data)
fig.add_traces(smoothed_data_gauss)
fig.update_layout(
    width=1200,
    height=700,
    title="STAND3 coincidence 1000"
)
fig.show()

In [ ]:
def find_extrema(time_series):
    """
    Identify local maxima and minima in a time series.
    
    Args:
    - time_series (array-like): The time series to analyze.
    
    Returns:
    - maxima (list): Indices of local maxima.
    - minima (list): Indices of local minima.
    """
    time_series = np.array(time_series)
    n = len(time_series)
    
    # Lists to store indices of local maxima and minima
    maxima = []
    minima = []
    
    # Loop through the time series to find extrema
    for i in range(1, n - 1):
        # Local Maximum Condition: time_series[i] > time_series[i-1] and time_series[i] > time_series[i+1]
        if time_series[i] > time_series[i - 1] and time_series[i] > time_series[i + 1]:
            maxima.append(i)
        
        # Local Minimum Condition: time_series[i] < time_series[i-1] and time_series[i] < time_series[i+1]
        elif time_series[i] < time_series[i - 1] and time_series[i] < time_series[i + 1]:
            minima.append(i)
    
    return maxima, minima

def mark_extrema(time_series):
    """
    Mark the local maxima and minima in a time series by returning a list with flags.
    A positive value indicates a local maximum, and a negative value indicates a local minimum.
    
    Args:
    - time_series (array-like): The time series to analyze.
    
    Returns:
    - marked_series (list): A list where 1 indicates local maxima, -1 indicates local minima, and 0 otherwise.
    """
    maxima, minima = find_extrema(time_series)
    
    # Create an array to mark the extrema
    marked_series = np.zeros(len(time_series), dtype=int)
    
    # Mark the local maxima
    for max_idx in maxima:
        marked_series[max_idx] = 1
    
    # Mark the local minima
    for min_idx in minima:
        marked_series[min_idx] = -1
    
    return marked_series

In [ ]:
stand3_1000_df["extrema"] = mark_extrema(stand3_1000_df.gaussian_smoothed)

In [ ]:
stand3_1000_df["min_points"] = [stand3_1000_df.gaussian_smoothed[i] if stand3_1000_df.extrema[i] < 0 else np.nan for i in stand3_1000_df.index]

In [ ]:
def assign_interval_serial_numbers(time_series):
    """
    Assign serial numbers to intervals defined by consecutive local minima in a time series.
    
    Args:
    - time_series (array-like): The time series to analyze.
    
    Returns:
    - df (pandas DataFrame): A DataFrame with the original time series and a serial number column.
    """
    # Find the local minima
    _, minima = find_extrema(time_series)
    #print(minima)

    # Create a column for the serial number of the intervals
    interval_serials = np.zeros(len(time_series), dtype=int)
    # Assign serial numbers based on intervals between local minima
    serial_number = 1
    for i in range(1, len(minima)):

        # Get the interval between consecutive minima
        start_idx = minima[i - 1]
        end_idx = minima[i]
        
        # Assign the serial number to all points in this interval
        interval_serials[start_idx:end_idx] = serial_number
        serial_number += 1
    return interval_serials

In [ ]:
stand3_1000_df["interval_number"] = assign_interval_serial_numbers(stand3_1000_df.gaussian_smoothed)

In [ ]:
interval_df = stand3_1000_df.groupby("interval_number")["STAND3_coincidence_1000"].agg(["max", "min"])
threshold = 0.1
interval_df["min_max_diff"] = interval_df["max"]/interval_df["min"] >= 1 + threshold
interval_df_10_percents = interval_df[interval_df["min_max_diff"]]

In [ ]:
len(interval_df_10_percents)

In [ ]:
interval_df_10_percents.index[40:60]

In [ ]:
# Example interval
sample = stand3_1000_df[stand3_1000_df.interval_number.isin([5411])]

In [ ]:
fig = go.Figure()
raw_data = go.Scatter(
    x=sample.timestamp,
    y=sample.STAND3_coincidence_1000,
    mode="lines",
    name="raw data",
    )
markers = go.Scatter(
    x=sample.timestamp,
    y=sample.min_points,
    mode="markers",
    name="interval borders",
    )
smoothed_data_gauss = go.Scatter(
    x=sample.timestamp,
    y=sample.gaussian_smoothed,
    mode="lines",
    name="double gauss smooth"
    )
fig.add_traces(raw_data)
fig.add_traces(markers)
fig.add_traces(smoothed_data_gauss)
fig.update_layout(
    width=1200,
    height=700,
    title="STAND3 coincidence 1000"
)
fig.show()